# Validate the annotated datasets

This notebook provides code for validating the evaluation set .txt files and retrieving information about the anthropomorphic components.

1. Make sure that no sentence was annotated with conflicting annotations
2. Make sure that there are no duplicate sentences in a sentence
3. Make sure that the .txt files used to create the evaluation sets are well-formed - i.e, the IDs contain the database prefix (used to locate them in the dataframe) and that each row contains exactly seven tab-separated values.
4. Check that the annotations are correct - e.g. the positive set contains only ['p1','p2','p3'] scores, the negative set contains only ['n1','n2','n3'] scores, and the inconclusive set has only 'inc'.
5. Retrieving the anthropomorphic components
6. Retrieving the AI entity lemmas (i.e. without descriptors and modifiers)

In [74]:
import re
        
def get_sentences_dict(filename,score):   

    sentences_dict = {}
    duplicate_ids = []
    duplicate_sentence_pairs = []
    
    sentences = open(f"../preprocessed_data/evaluation_sentences/{filename}_{score}.txt","r")
    
    for line in sentences.readlines():
        line = line.strip()
        line = line.split("\t")
        if len(line) == 0:
            break
        sent_id = line[0]
        sent_info = line[1:]

        # wellformedness checks
        if len(line) != 7:
            print(f"The row with the ID {sent_id} in {filename}_{score}.txt is not well-formed.")
        id_prefix = sent_id[:6]
        if not re.match(r"^[1-5]{1}_(arx|acl)_", id_prefix):
            print(f"The ID {sent_id} in {filename}_{score} is not well-formed.")
        
        if sent_id not in sentences_dict:
            if sent_info not in sentences_dict.values():
                sentences_dict[sent_id] = sent_info
            else: # the sentence appears twice with different IDs 
                other_id = [key for key in sentences_dict if sentences_dict[key] == sent][0]
                duplicate_sentence = (other_id,sent_id)
                duplicate_sentence_pairs.append(duplicate_sentence) 
        else: # the sentence appears twice with the same ID
            duplicate_ids.append(sent_id)

    return sentences_dict,duplicate_ids,duplicate_sentence_pairs

def get_anthro_components(filename,score):

    anthro_components = {}

    print(f"Retrieving a list of anthropomorphic words in {filename}_{score}.txt:")
    
    sentences_dict = get_sentences_dict(filename,score)[0]
    all_sentences_info = sentences_dict.values()
    
    for sent_info in all_sentences_info:
        if sent_info[4] not in anthro_components:
            anthro_components[sent_info[4]] = 1
        else:
            anthro_components[sent_info[4]] += 1

    return anthro_components

def get_ai_components(filename,score):

    ai_components = {}

    print(f"Retrieving a list of anthropomorphic words in {filename}_{score}.txt:")
    
    sentences_dict = get_sentences_dict(filename,score)[0]
    all_sentences_info = sentences_dict.values()
    
    for sent_info in all_sentences_info:
        if sent_info[3] not in ai_components:
            ai_components[sent_info[3]] = 1
        else:
            ai_components[sent_info[3]] += 1

    return ai_components

def check_annotations(filename,score):

    annotations_dict = {}

    if score == 'positive':
        annotations = ['p1','p2','p3']
    elif score == 'negative':
        annotations = ['n1','n2','n3']
    elif score == 'inconclusive':
        annotations = ['inc']

    print(f"Checking annotations in {filename}_{score}.txt:")

    sentences_dict = get_sentences_dict(filename,score)[0]
    all_sentences_info = sentences_dict.values()

    for sent_id,sent_info in sentences_dict.items():
        if sent_info[5] not in annotations:
            print(f"Fix incorrect annotation {sent_info[5]} in the sentence with the ID {sent_id}")
        if sent_info[5] not in annotations_dict:
            annotations_dict[sent_info[5]] = 1
        else:
            annotations_dict[sent_info[5]] += 1

    return annotations_dict

In [77]:
def pairwise_conflict_check(filename,case1,case2):

    score1 = case1[0] # positive, negative or inconclusive
    num1 = case1[1] # number of cases in file

    score2 = case2[0] # positive, negative or inconclusive
    num2 = case2[1] # number of cases in file

    print(f"Comparing {score1} cases and {score2} cases for the {filename} set...")

    conflicting_annotation = False

    dict1 = get_sentences_dict(filename,score1)[0]
    dict2 = get_sentences_dict(filename,score2)[0]

    for id1,sent in dict1.items():
        if id1 in dict2:
            conflicting_annotation = True
            print(f"The {score1} sentence with the ID ",id1,f" appears in the {score2} set with the same ID")
        elif sent in dict2.values():
            conflicting_annotation = True
            id2 = [key for keys in dict2.keys() if dict2[key] == sent][0]
            print(f"The {score1} sentence with the ID  ",id1,
                  f" appears in the {score2} set with the ID ",id2)

    return conflicting_annotation

def validate_num_and_duplicates(filename,case):

    score = case[0] # positive, negative or inconclusive
    num = case[1] # number of cases in file

    duplicates = False
    print(f"Checking for duplicate entries in {filename}_{score}.txt...")

    sentences_dict = get_sentences_dict(filename,score)[0]
    duplicate_ids = get_sentences_dict(filename,score)[1]
    duplicate_sentence_pairs = get_sentences_dict(filename,score)[2]
    
    if len(sentences_dict.keys()) > num:
        print(f"There are more than {num} sentences in {filename}_{score}.txt.")
    elif len(sentences_dict.keys()) < num:
        print(f"There are less than {num} sentences in {filename}_{score}.txt.")

    if duplicate_ids:
        duplicates = True
        print("The sentences with the following ids appear twice: ",duplicate_ids,
             f" in {filename}_{score}.txt")

    if duplicate_sentence_pairs:
        duplicates = True
        print("The following ID pairs refer to the same sentence: ",duplicate_sentence_pairs,
             f" in {filename}_{score}.txt")

    return duplicates

def check_conflicting_annotations(cat,cases_and_nums):

    num_pos = cases_and_nums["positive"] 
    num_neg = cases_and_nums["negative"]
    num_inc = cases_and_nums["inconclusive"]

    conflicting_pos_neg_annotations = pairwise_conflict_check(cat,("positive",num_pos),("negative",num_neg))
    conflicting_pos_inc_annotations = pairwise_conflict_check(cat,("positive",num_pos),("inconclusive",num_inc))
    conflicting_neg_inc_annotations = pairwise_conflict_check(cat,("negative",num_neg),("inconclusive",num_inc))

    if conflicting_pos_neg_annotations:
        print("Resolve conflicting annotations in the positive and negative sets!!!")
    elif conflicting_pos_inc_annotations:
        print("Resolve conflicting annotations in the positive and inconclusive sets!!!")
    elif conflicting_neg_inc_annotations:
        print("Resolve conflicting annotations in the negative and inconclusive sets!!!")
    else:
        print("No conflicting annotations.")

    return

def check_for_duplicates(cat,cases_and_nums):

    num_pos = cases_and_nums["positive"] 
    num_neg = cases_and_nums["negative"]
    num_inc = cases_and_nums["inconclusive"]

    duplicates_in_pos = validate_num_and_duplicates(cat,("positive",num_pos))
    duplicates_in_neg = validate_num_and_duplicates(cat,("negative",num_neg))
    duplicates_in_inc = validate_num_and_duplicates(cat,("inconclusive",num_inc))

    if duplicates_in_pos:
        print("Resolve duplicates in the positive set!!!")
    elif duplicates_in_neg:
        print("Resolve duplicates in the negative set!!!")
    elif duplicates_in_inc:
        print("Resolve duplicates in the inconclusive set!!!")
    else:
        print("No duplicate utterances.")

    return


#### Check sentences for each category

change parameter of get_sentences. The options are:
1. verb_subjects - sentences in which the AI entity is the subject of an anthropomorphic verb (nsubj)
2. verb_objects - sentences in which the AI entity is object of an anthropomorphic verb (pobj,dobj)
4. adjective_phrases - sentences in which the AI entity is part of an anthropomorphic adjectival phrase
5. noun_phrases - sentences in which the AI entity is part of an anthropomorphic noun phrase
6. possessives - sentences in which the AI entity is immediately followed by a possessive marker
7. comparisons - sentences in which the AI entity is being compared to humans explicitly

In [82]:
def run_tests(cat,cases_and_nums):

    check_for_duplicates(category,cases_and_nums)
    print()
    check_conflicting_annotations(category,cases_and_nums)
    print()

    return

cases_and_nums = {"positive":60,"negative":60,"inconclusive":30}
category = "verb_subjects"

run_tests(category,cases_and_nums)
for case in cases_and_nums: 
    print(check_annotations(category,case))
    print()
    anthro_components = get_anthro_components(category,case)
    print(anthro_components)
    print()
    ai_components = get_ai_components(category,case)
    print(ai_components)
    print()

Checking for duplicate entries in verb_subjects_positive.txt...
Checking for duplicate entries in verb_subjects_negative.txt...
Checking for duplicate entries in verb_subjects_inconclusive.txt...
No duplicate utterances.

Comparing positive cases and negative cases for the verb_subjects set...
Comparing positive cases and inconclusive cases for the verb_subjects set...
Comparing negative cases and inconclusive cases for the verb_subjects set...
No conflicting annotations.

Checking annotations in verb_subjects_positive.txt:
{'p3': 15, 'p2': 41, 'p1': 4}

Retrieving a list of anthropomorphic words in verb_subjects_positive.txt:
{'determine when to request help': 1, 'understand': 2, 'recognize': 1, 'think': 1, 'infer': 2, 'resolve a conflict': 1, 'learn from each other': 1, 'collaborate': 1, 'exhibit overconfidence': 1, 'memorize': 2, 'remember': 2, 'see': 1, 'recall': 1, 'have awareness': 1, 'decide to trust': 1, 'suffer': 1, 'believe': 1, 'understand, intepret and predict': 1, 'try to 